In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [31]:
cookies = {
    '_ga': 'GA1.1.2121071378.1694150502',
    '_ga_HJJM6M6K85': 'GS1.1.1694162912.2.1.1694162915.0.0.0',
}

headers = {
    'authority': 'blog.pandai.org',
    'accept': '*/*',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': '_ga=GA1.1.2121071378.1694150502; _ga_HJJM6M6K85=GS1.1.1694162912.2.1.1694162915.0.0.0',
    'origin': 'https://blog.pandai.org',
    'pragma': 'no-cache',
    'referer': 'https://blog.pandai.org/',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

In [36]:
def generator():
    while True:
        yield
        
index = 1
all_a = []

for _ in tqdm(generator()):
    data = {
        'action': 'saxon_loadmore',
        'query[posts_per_page]': '100',
        'query[offset]': '',
        'query[orderby]': 'date',
        'query[order]': 'DESC',
        'query[cat]': '',
        'query[post_type]': 'post',
        'query[post_status]': 'publish',
        'query[ignore_sticky_posts]': 'true',
        'query[suppress_filters]': '0',
        'query[error]': '',
        'query[m]': '',
        'query[p]': '0',
        'query[post_parent]': '',
        'query[subpost]': '',
        'query[subpost_id]': '',
        'query[attachment]': '',
        'query[attachment_id]': '0',
        'query[name]': '',
        'query[pagename]': '',
        'query[page_id]': '0',
        'query[second]': '',
        'query[minute]': '',
        'query[hour]': '',
        'query[day]': '0',
        'query[monthnum]': '0',
        'query[year]': '0',
        'query[w]': '0',
        'query[category_name]': '',
        'query[tag]': '',
        'query[tag_id]': '',
        'query[author]': '',
        'query[author_name]': '',
        'query[feed]': '',
        'query[tb]': '',
        'query[paged]': '0',
        'query[meta_key]': '',
        'query[meta_value]': '',
        'query[preview]': '',
        'query[s]': '',
        'query[sentence]': '',
        'query[title]': '',
        'query[fields]': '',
        'query[menu_order]': '',
        'query[embed]': '',
        'query[cache_results]': 'true',
        'query[update_post_term_cache]': 'true',
        'query[update_menu_item_cache]': 'false',
        'query[lazy_load_term_meta]': 'true',
        'query[update_post_meta_cache]': 'true',
        'query[nopaging]': 'false',
        'query[comments_per_page]': '50',
        'query[no_found_rows]': 'false',
        'page': str(index),
        'block': 'blog',
        'post_template': 'blog',
    }

    response = requests.post('https://blog.pandai.org/wp-admin/admin-ajax.php', cookies=cookies, headers=headers, data=data)
    soup = BeautifulSoup(response.content, "lxml")
    a = soup.find_all('article')
    a = [a_.find('a').get('href') for a_ in a]
    
    if not len(a):
        break
        
    all_a.extend(a)
    
    index += 1

3it [00:03,  1.17s/it]


In [37]:
len(all_a)

233

In [39]:
urls = sorted(list(set(all_a)))
len(urls)

233

In [40]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

In [42]:
soup.find('h1').text

'10 Class Activities That Will Keep Your Students Engaged'

In [44]:
soup.find('div', {'class': 'entry-content'}).get_text(separator="\n")

'\n\nAs a teacher, it’s your job to make sure your students are engaged in the material. But sometimes, no matter how interesting you make the subject matter, there are always a few students who just can’t seem to stay focused.\xa0\n\n\nIf you’re struggling to keep your students engaged, don’t worry—you’re not alone. We’ve compiled a list of 10 tried-and-true class activities that are sure to keep even the most disengaged student involved. From group work to hands-on learning, these activities will get your students excited about learning again.\n\n\nGet Students Up And Moving With An Activity That Gets Them Out Of Their Seats\n\n\n\n\nEngaging your students in active learning is a great way to keep them focused and engaged in the lesson. There are many different ways to get students up and moving, so find one that works best for you and your students. For example, you could have them stand up and do a quick stretch every 10 minutes, or have them work in pairs to discuss a question. Yo

In [45]:
data = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    data.append({
        'url': url,
        'title': soup.find('h1').text.strip(),
        'body': soup.find('div', {'class': 'entry-content'}).get_text(separator="\n"),
    })

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 233/233 [02:02<00:00,  1.90it/s]


In [46]:
with open('blog.pandai.com', 'w') as fopen:
    for d in data:
        fopen.write(f'{json.dumps(d)}\n')